In [117]:
import os
from datetime import datetime
import pandas as pd
import collections
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA

In [118]:
EPSILON = 0.01

In [119]:
class NooksExperiment:
    
    def _init_graphs(self):
        self.network_graph = nx.Graph()
        self.nooks_graph = nx.Graph()
        
        nft = len(self.member_vectors[0])
        self.graph_colors = []
        
        if nft == 1:
            color = self.member_vectors
        else:
            pca = PCA(n_components=1)
            color = pca.fit_transform(self.member_vectors)
        
        self.graph_colors = color.reshape(-1)
            
        for member in range(self.total_members):
            self.network_graph.add_node(member)
            self.nooks_graph.add_node(member)
        
    def __init__(self, member_vectors, alpha):
        self.total_members = len(member_vectors)
        self.member_vectors = member_vectors
        self.interacted = np.zeros((self.total_members, self.total_members))
        self.all_interactions = np.zeros((self.total_members, self.total_members))
        self.alpha = alpha
        self._init_graphs()
        
        self.network_output_folder = "Results/" + datetime.now().strftime("%Y_%m_%d-%I_%M_%S_%p") + "/Network/"
        self.nooks_output_folder = "Results/" + datetime.now().strftime("%Y_%m_%d-%I_%M_%S_%p") + "/Nooks/"
        self.info_output_folder = "Results/" + datetime.now().strftime("%Y_%m_%d-%I_%M_%S_%p") + "/Info/" 
        os.makedirs(self.network_output_folder)
        os.makedirs(self.nooks_output_folder)
        os.makedirs(self.info_output_folder)
        
        
    def reset(self):
        self.interacted = np.zeros((self.total_members, self.total_members))
    
    def _update_interacted(self, member_allocs, nooks_allocs):
        for member in range(self.total_members):
            if not member in member_allocs:
                continue
            self.interacted[member] += nooks_allocs[member_allocs[member]]
            self.all_interactions[member] += nooks_allocs[member_allocs[member]]
        
    # TODO see if running median is needed; space & time 
    def _create_nook_allocs(self, num_nooks, nook_creators, nook_swipes):
        
        nooks_allocs = np.zeros((num_nooks, self.total_members))
        member_allocs = {}
        nooks_mem_cnt = np.ones((num_nooks))
        nooks_mem_int_cnt = np.zeros((num_nooks, self.total_members))
                            
        
        for nook in range(num_nooks):
            nooks_allocs[nook][nook_creators[nook]] = 1
            member_allocs[nook_creators[nook]] = nook
            #nooks_mem_cnt[nook] = 1
            
            
        for member in range(self.total_members):
            if member in member_allocs or not(np.sum(nook_swipes[member])):
                continue
            
            swipes = nook_swipes[member]
            median_reps = []
            
            for nook in range(num_nooks):
                if not nook_swipes[member][nook]:
                    median_reps.append(1) # this value will be ignored
                    continue
                median_rep = np.median(self.member_vectors[nooks_allocs[nook]==1], axis=0)
                median_reps.append(np.linalg.norm(self.member_vectors[member]-median_rep))
            
            median_reps = 1/(EPSILON + np.array(median_reps))
            
            interacted_by = nooks_mem_int_cnt[:, member]
            wts = (EPSILON + interacted_by)/(nooks_mem_cnt * (1 + (self.alpha * median_reps)))
            sel_wts = wts * nook_swipes[member]
            total_sel_wts = np.sum(sel_wts)
            selected_nook = np.random.choice(num_nooks, p=sel_wts/total_sel_wts)
            
            nooks_allocs[selected_nook][member] = 1
            member_allocs[member] = selected_nook
            nooks_mem_cnt[selected_nook] += 1
            nooks_mem_int_cnt += self.interacted[member]
            
        self._update_interacted(member_allocs, nooks_allocs)
        return member_allocs, nooks_allocs

    def _update_nooks_graph(self, member_allocs, day=0):
        print("______NOOKS_________" )
        self.nooks_graph.remove_edges_from(list(self.nooks_graph.edges()))
        for m1 in range(self.total_members):
            if m1 not in member_allocs:
                continue
            for m2 in range(m1):
                if m2 not in member_allocs:
                    continue
                if member_allocs[m1] == member_allocs[m2]:
                    self.nooks_graph.add_edge(m1, m2, weight=self.all_interactions[m1, m2])
        weights = [self.nooks_graph[u][v]['weight'] for u,v in self.nooks_graph.edges()]
        nx.draw(self.nooks_graph, node_color=self.graph_colors, width=weights)
        print("DAY " + str(day))
        plt.show()
        #plt.savefig(os.path.join(self.nooks_output_folder, "day_"+ str(day) + ".png"))
        plt.close()
        
    def _update_network_graph(self, member_allocs, day=0, print_summary=False):
        print("______NETWORK_________" )
        self.network_graph.remove_edges_from(list(self.network_graph.edges()))
        for m1 in range(self.total_members):
            for m2 in range(m1):
                if self.all_interactions[m1, m2]:
                    self.network_graph.add_edge(m1, m2, weight=self.all_interactions[m1, m2])
                    
        weights = [self.network_graph[u][v]['weight'] for u,v in self.network_graph.edges()]
        nx.draw(self.network_graph, node_color=self.graph_colors, width=weights)
        plt.show()
        plt.close()
        if not print_summary:
            return
        
        top_neighbors = []
        for n in range(self.total_members):
            neighbors = np.array([self.member_vectors[m] for m in self.network_graph.neighbors(n) if not(m==n)])
            neighbors = neighbors.reshape(-1)
            edge_weights = np.array([self.network_graph.edges[n, m]['weight'] for m in self.network_graph.neighbors(n) if not(m==n)])
            
            #print(weighted_neighbors.shape, self.member_vectors.shape)
            total_weights = np.sum(edge_weights)
            edge_weights_norm = (edge_weights/total_weights).reshape(-1)
            print(edge_weights_norm * neighbors)

            mean_rep = np.sum(a=edge_weights_norm * neighbors)
            
            top_neighbors.append([self.member_vectors[n], mean_rep])
            #print(self.member_vectors[n], [self.member_vectors[m] for m in top_neighbors])
        df = pd.DataFrame(top_neighbors)
        df.to_csv(os.path.join(self.info_output_folder, str(day)))
        print(df)
        #nx.write_gexf(self.network_graph, "test.gexf")
        #plt.savefig(os.path.join(self.network_output_folder, "day_"+ str(day) + ".png"))
                             
                                                
    def _update_graphs(self, member_allocs, day=0, print_summary=False):
        self._update_network_graph(member_allocs, day, print_summary=print_summary)
        self._update_nooks_graph(member_allocs, day)
    
    # assumes we only pop off atmost 1 story created by a member
    def _create_nooks(self, num_nooks):
        return np.random.choice(self.total_members, size=num_nooks, replace=False)
        
        
    def _create_random_swipes(self, num_nooks):
        return np.random.randint(2, size=(self.total_members, num_nooks))
            
    
    def run(self, num_days=28, reset_time=7, graph_time=1, num_nooks_mode='uniform', **kwargs):
        if num_nooks_mode not in ['uniform', 'random']:
            raise "Experiment run mode not implemented"
            return 
        if num_nooks_mode == 'uniform':
            num_nooks = kwargs["num_nooks"]
        for i in range(num_days):
            print_summary = False
            if not i % reset_time:
                print_summary = True
                self.reset()
                            
            if num_nooks_mode == 'random':
                num_nooks = np.random.randint(kwargs["min_nooks"], kwargs["max_nooks"])

            nook_creators = self._create_nooks(num_nooks) 
            nook_swipes = self._create_random_swipes(num_nooks)
            member_allocs, nooks_allocs = self._create_nook_allocs(num_nooks, nook_creators, nook_swipes)
            
            if not i % graph_time:
                self._update_graphs(member_allocs=member_allocs, day=i, print_summary=print_summary)
            
            
            
        

In [120]:
member_vectors = np.random.random(size=(100, 1))
exp = NooksExperiment(member_vectors, alpha=500)
exp.run(num_nooks=5)



FileNotFoundError: [Errno 2] No such file or directory: 'Results'